In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
train_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/train_hypothesis_evidences.csv')
train_df

,hypothesis,evidences,label
0,All the primary trial participants do not rece...,"[""INTERVENTION 1: "", "" Diagnostic (FLT PET)"",...",Contradiction
1,"Patients with Platelet count over 100,000/mm¬¨...","["" PATIENT CHARACTERISTICS:"", "" ANC 1,500/m...",Contradiction
2,Heart-related adverse events were recorded in ...,"[""Adverse Events 1:"", "" Supraventricular tach...",Entailment
3,Adult Patients with histologic confirmation of...,"[""Inclusion Criteria:"", "" Patients with histo...",Contradiction
4,Laser Therapy is in each cohort of the primary...,"[""INTERVENTION 1: "", "" Laser Therapy Alone"", ...",Contradiction
...,...,...,...
1695,"Adequate blood, kidney, and hepatic function a...","[""Inclusion Criteria:"", "" Postmenopausal wome...",Entailment
1696,The Ridaforolimus + Dalotuzumab + Exemestane g...,"[""Outcome Measurement: "", "" 1. Progression-fr...",Contradiction
1697,The only difference between the interventions ...,"[""INTERVENTION 1: "", "" Prone"", ""Prone positio...",Entailment
1698,Patients must have a white blood cell count ab...,"["" WBC > 1,500/mm\u00b3""]",Entailment


In [3]:
hypothesis_lst=train_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=train_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=train_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

1700

In [4]:
import torch
import numpy as np
import random
import math
from torch.utils.data import Dataset

class InputSequence(Dataset):
    def __init__(self, tok, l_text, l_text2, l_label=None, batch_size=64, gpu=True):
        self.data_len = len(l_text)
        self.data_idx = list(range(self.data_len))
        self.texts = tok(l_text, l_text2, padding=True, truncation=True, max_length=512, return_tensors='pt')
        self.l_label = np.array(l_label) if l_label is not None else None  # Initialize labels only if provided
        self.batch_size = batch_size
        self.gpu = gpu
        print('Tokenization done')

    def on_epoch_end(self):
        random.shuffle(self.data_idx)

    def __getitem__(self, i):
        start = i * self.batch_size
        batch_idx = self.data_idx[start:min(start + self.batch_size, self.data_len)]

        # Convert tensors to CUDA if GPU is enabled
        return_texts = {k: self.texts[k][batch_idx].cuda() if self.gpu else self.texts[k][batch_idx] for k in self.texts}

        if self.l_label is not None:  # Check if labels were provided
            return_labels = torch.from_numpy(self.l_label[batch_idx].astype(np.int64))
            if self.gpu:
                return_labels = return_labels.cuda()
            return return_texts, return_labels
        else:
            return return_texts  # Only return texts if no labels are available

    def __len__(self):
        return math.ceil(self.data_len / self.batch_size)


In [5]:
import random
import math
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

text_tok = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
text_clf = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-base-cased-v1.2")
text_clf.num_parameters()

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


108311810

In [7]:
training_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)
len(training_data)

Tokenization done


27

In [8]:
dev_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/dev_hypothesis_evidences.csv')
dev_df

,hypothesis,evidences,label
0,there is a 13.2% difference between the result...,"[""Outcome Measurement: "", "" Event-free Surviv...",Contradiction
1,Patients with significantly elevated ejection ...,"["" Cardiac left ventricular function with res...",Contradiction
2,a significant number of the participants in th...,"["" Enterocolitis 1/167 (0.60%)"", "" Enterocol...",Contradiction
3,the primary trial does not report the PFS or o...,"[""Outcome Measurement: "", "" Local Control Usi...",Entailment
4,Prior treatment with fulvestrant or with a pho...,"["" Prior treatment with a phosphatidylinosito...",Contradiction
...,...,...,...
195,The the primary trial intervention involves on...,"[""INTERVENTION 1: "", "" Letrozole"", "" Partici...",Contradiction
196,the secondary trial reported 1 single case of ...,"[""Adverse Events 1:"", "" Total: 16/48 (33.33%)...",Entailment
197,the secondary trial and the primary trial do n...,"[""Outcome Measurement: "", "" Number of Patient...",Entailment
198,the outcome measurement of the primary trial i...,"[""Outcome Measurement: "", "" Progression-free ...",Entailment


In [9]:
hypothesis_lst=dev_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=dev_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=dev_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

200

In [10]:
dev_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

Tokenization done


In [11]:
class Model(nn.Module):
    def __init__(self,clf):
        super(Model, self).__init__()
        self.clf=clf
        self.loss=nn.CrossEntropyLoss()

    def forward(self, texts, labels, gpu=True):

        loss=self.loss(self.clf(**texts).logits, labels)

        return loss

In [12]:
model=Model(text_clf)

In [13]:
import os
import shutil
import torch
from torch import nn
from transformers import AdamW
import numpy as np
import math
import random

bat_s = 16
l_rate = 1e-5

training_data.batch_size = bat_s

model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=l_rate)
total_epoch_num = 10

for epoch in range(total_epoch_num):
    training_data.on_epoch_end()
    loss_sum = 0.0
    loss_count = 0
    model.train()  # Ensure model is in training mode
    
    for batch in range(len(training_data)):
        optimizer.zero_grad()
        batch_texts, batch_labels = training_data[batch]
        loss_count += len(batch_texts['input_ids'])
        loss = model(batch_texts, batch_labels)
        print('epoch:', epoch, 'batch:', batch, 'loss:', loss.item(), end='\n' if batch == 0 or batch + 1 == len(training_data) or (batch + 1) % 1000 == 0 else '\r')
        loss_sum += loss.item() * len(batch_texts['input_ids'])
        loss.backward()
        optimizer.step()
        
    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_loss_sum = 0.0
    val_loss_count = 0
    with torch.no_grad():
        for batch in range(len(dev_data)):
            batch_texts, batch_labels = dev_data[batch]
            val_loss_count += len(batch_texts['input_ids'])
            val_loss = model(batch_texts, batch_labels)
            val_loss_sum += val_loss.item() * len(batch_texts['input_ids'])

    avg_val_loss = val_loss_sum / val_loss_count
    print(f'Validation - Epoch: {epoch}, Avg Loss: {avg_val_loss}')

    # Save the model after each epoch
    model_path = f'./clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{epoch:05d}.pt'
    model.clf.save_pretrained(model_path)
    print(f'Model saved to {model_path}')

_ = model.cpu()  # Move model back to CPU if necessary

epoch: 0 batch: 0 loss: 0.64802086353302
epoch: 0 batch: 106 loss: 0.7162390947341919
Validation - Epoch: 0, Avg Loss: 0.694146432876587
Model saved to ./clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_00000.pt
epoch: 1 batch: 0 loss: 0.6931287050247192
epoch: 1 batch: 106 loss: 0.7450661063194275
Validation - Epoch: 1, Avg Loss: 0.6926659679412842
Model saved to ./clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_00001.pt
epoch: 2 batch: 0 loss: 0.6728920936584473
epoch: 2 batch: 106 loss: 0.8213199973106384
Validation - Epoch: 2, Avg Loss: 0.69583087682724
Model saved to ./clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_00002.pt
epoch: 3 batch: 0 loss: 0.7436635494232178
epoch: 3 batch: 106 loss: 0.6952358484268188
Validation - Epoch: 3, Avg Loss: 0.6885556483268738
Model saved to ./clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_00003.pt
epoch: 4 batch: 0 loss: 0.6733831763267517
epoch: 4 batch: 106 loss: 0.7267605066299438
Validation - Epoch: 4, Avg Loss: 0.676571147441864
Mo

In [14]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(dev_data)):
            batch_texts, batch_labels = dev_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.508716,0.516432,0.486726,0.55,0.485,0.414062,0.585938
1,1,0.533684,0.666667,0.500000,1.00,0.500,0.492188,0.507812
2,2,0.533392,0.535211,0.504425,0.57,0.505,0.546875,0.453125
3,3,0.541965,0.666667,0.500000,1.00,0.500,0.492188,0.507812
4,4,0.553566,0.633205,0.515723,0.82,0.525,0.484375,0.515625
5,5,0.635853,0.645669,0.532468,0.82,0.550,0.500000,0.500000
6,6,0.650941,0.659341,0.520231,0.90,0.535,0.468750,0.531250
7,7,0.653969,0.616740,0.551181,0.70,0.565,0.476562,0.523438
8,8,0.690757,0.650407,0.547945,0.80,0.570,0.492188,0.507812
9,9,0.667787,0.548780,0.703125,0.45,0.630,0.460938,0.539062


## Above results are for dev data

In [16]:
numerical_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Numerical_Statements_hypothesis_evidences.csv')
print(numerical_df)

hypothesis_lst=numerical_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=numerical_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=numerical_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

numerical_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   there is a 13.2% difference between the result...   
1   Patients with significantly elevated ejection ...   
2   Prior treatment with fulvestrant or with a pho...   
3   only patients with a HER2-positive status can ...   
4   the shortest PFS in cohort 1 of the primary tr...   
..                                                ...   
88  All the primary trial patients achieved either...   
89  A patient with a node positive T2 N2 M0 adenoc...   
90  the primary trial results show that the Trastu...   
91  the secondary trial reported 1 single case of ...   
92  All the primary trial patients had a minimum o...   

                                            evidences          label  
0   ["Outcome Measurement: ", "  Event-free Surviv...  Contradiction  
1   ["  Cardiac left ventricular function with res...  Contradiction  
2   ["  Prior treatment with a phosphatidylinosito...  Contradiction  
3   ["  HER2-positive status (p

In [17]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(numerical_data)):
            batch_texts, batch_labels = numerical_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.466792,0.617886,0.469136,0.904762,0.494624,0.578125,0.421875
1,1,0.508319,0.622222,0.451613,1.000000,0.451613,0.515625,0.484375
2,2,0.440028,0.468085,0.423077,0.523810,0.462366,0.546875,0.453125
3,3,0.486060,0.622222,0.451613,1.000000,0.451613,0.515625,0.484375
4,4,0.487970,0.584071,0.464789,0.785714,0.494624,0.515625,0.484375
5,5,0.538937,0.630631,0.507246,0.833333,0.559140,0.468750,0.531250
6,6,0.523688,0.600000,0.461538,0.857143,0.483871,0.515625,0.484375
7,7,0.526483,0.527473,0.489796,0.571429,0.537634,0.406250,0.593750
8,8,0.611952,0.581818,0.470588,0.761905,0.505376,0.406250,0.593750
9,9,0.596912,0.393443,0.631579,0.285714,0.602151,0.500000,0.500000


## Above results are for numerical data

In [19]:
non_numerical_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Non_Numerical_Statements_hypothesis_evidences.csv')
print(non_numerical_df)

hypothesis_lst=non_numerical_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=non_numerical_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=non_numerical_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

non_numerical_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                            hypothesis  \
0    a significant number of the participants in th...   
1    the primary trial does not report the PFS or o...   
2    the outcome measurement of the primary trial i...   
3    The the primary trial results section reports ...   
4    both cohorts of the primary trial receive iden...   
..                                                 ...   
102  Patients must be bedbound or severaly disabled...   
103  the results of the secondary trial and the pri...   
104  The the primary trial intervention involves on...   
105  the secondary trial and the primary trial do n...   
106  the outcome measurement of the primary trial i...   

                                             evidences          label  
0    ["  Enterocolitis 1/167 (0.60%)", "  Enterocol...  Contradiction  
1    ["Outcome Measurement: ", "  Local Control Usi...     Entailment  
2    ["Outcome Measurement: ", "  Progression-free ...  Contradiction  
3    ["Outcome 

In [20]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(non_numerical_data)):
            batch_texts, batch_labels = non_numerical_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.562641,0.000000,0.000000,0.000000,0.457944,0.520000,0.480000
1,1,0.568111,0.703030,0.542056,1.000000,0.542056,0.480000,0.520000
2,2,0.616581,0.588235,0.573770,0.603448,0.542056,0.440000,0.560000
3,3,0.588396,0.703030,0.542056,1.000000,0.542056,0.480000,0.520000
4,4,0.615219,0.671233,0.556818,0.844828,0.551402,0.426667,0.573333
5,5,0.698396,0.657343,0.552941,0.810345,0.542056,0.440000,0.560000
6,6,0.726224,0.705882,0.568421,0.931034,0.579439,0.386667,0.613333
7,7,0.722123,0.676471,0.589744,0.793103,0.588785,0.426667,0.573333
8,8,0.745150,0.705882,0.615385,0.827586,0.626168,0.360000,0.640000
9,9,0.718417,0.640777,0.733333,0.568966,0.654206,0.386667,0.613333


## Above results are for Non numerical data

In [22]:
single_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Single_hypothesis_evidences.csv')
print(single_df)

hypothesis_lst=single_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=single_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=single_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

single_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                            hypothesis  \
0    there is a 13.2% difference between the result...   
1    the primary trial does not report the PFS or o...   
2    Prior treatment with fulvestrant or with a pho...   
3    only patients with a HER2-positive status can ...   
4    the shortest PFS in cohort 1 of the primary tr...   
..                                                 ...   
135  A patient with a node positive T2 N2 M0 adenoc...   
136  Patients must be bedbound or severaly disabled...   
137  the primary trial results show that the Trastu...   
138  the outcome measurement of the primary trial i...   
139  All the primary trial patients had a minimum o...   

                                             evidences          label  
0    ["Outcome Measurement: ", "  Event-free Surviv...  Contradiction  
1    ["Outcome Measurement: ", "  Local Control Usi...     Entailment  
2    ["  Prior treatment with a phosphatidylinosito...  Contradiction  
3    ["  HER2-p

In [23]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(single_data)):
            batch_texts, batch_labels = single_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.515581,0.609195,0.509615,0.757143,0.514286,0.427083,0.572917
1,1,0.542686,0.666667,0.500000,1.000000,0.500000,0.458333,0.541667
2,2,0.557420,0.530612,0.506494,0.557143,0.507143,0.385417,0.614583
3,3,0.556162,0.666667,0.500000,1.000000,0.500000,0.458333,0.541667
4,4,0.560455,0.645503,0.512605,0.871429,0.521429,0.500000,0.500000
5,5,0.604035,0.637838,0.513043,0.842857,0.521429,0.541667,0.458333
6,6,0.606682,0.649485,0.508065,0.900000,0.514286,0.500000,0.500000
7,7,0.628146,0.634731,0.546392,0.757143,0.564286,0.510417,0.489583
8,8,0.658606,0.659459,0.530435,0.871429,0.550000,0.500000,0.500000
9,9,0.643733,0.542373,0.666667,0.457143,0.614286,0.468750,0.531250


## Above results are for Single Type data

In [25]:
comparison_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Comparison_hypothesis_evidences.csv')
print(comparison_df)

hypothesis_lst=comparison_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=comparison_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=comparison_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

comparison_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Patients with significantly elevated ejection ...   
1   a significant number of the participants in th...   
2   Diarrhoea is the most common adverse event in ...   
3   most participants in the secondary trial and t...   
4   Presence of hot flashes for at least 4 month p...   
5   the secondary trial and the primary trial both...   
6   the primary trial and the secondary trial use ...   
7   the primary trial and the secondary trial do n...   
8   the secondary trial and the primary trial do n...   
9   Between the secondary trial and the primary tr...   
10  All patients in the primary trial receive high...   
11  all participants of the primary trial have sta...   
12  The results from the secondary trial and the p...   
13  the primary trial and the secondary trial part...   
14  There are more cases of Urosepsis in the prima...   
15  the primary trial participants are not adminis...   
16  All the primary trial parti

In [26]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(comparison_data)):
            batch_texts, batch_labels = comparison_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.519701,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
1,1,0.541419,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
2,2,0.502462,0.545455,0.500000,0.600000,0.500000,0.500000,0.500000
3,3,0.530888,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
4,4,0.595833,0.600000,0.525000,0.700000,0.533333,0.466667,0.533333
5,5,0.717048,0.666667,0.589744,0.766667,0.616667,0.383333,0.616667
6,6,0.743195,0.683544,0.551020,0.900000,0.583333,0.416667,0.583333
7,7,0.709807,0.566667,0.566667,0.566667,0.566667,0.433333,0.566667
8,8,0.748067,0.622951,0.612903,0.633333,0.616667,0.383333,0.616667
9,9,0.727379,0.565217,0.812500,0.433333,0.666667,0.333333,0.666667


## Above results are for Comparison Type data

In [28]:
adverse_events_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/AdverseEvents_hypothesis_evidences.csv')
print(adverse_events_df)

hypothesis_lst=adverse_events_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=adverse_events_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=adverse_events_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

adverse_events_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   a significant number of the participants in th...   
1   The only case of congestive heart failure in t...   
2   Anaemia was the most common adverse event in c...   
3   The adverse events section in the primary tria...   
4   Diarrhoea is the most common adverse event in ...   
5   most participants in the secondary trial and t...   
6   The adverse events section in the primary tria...   
7   there were 4 types of Adverse events which did...   
8   None of the individual Aes recorded in the pri...   
9   The most common adverse event in the primary t...   
10  Febrile Neutropenia was the most common advers...   
11  None of the individual Aes recorded in the pri...   
12  the primary trial and the secondary trial do n...   
13  Between the secondary trial and the primary tr...   
14  Syncope was the most common adverse event reco...   
15  no cardiac or neural adverse events were recor...   
16  there was a higher percenta

In [29]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(adverse_events_data)):
            batch_texts, batch_labels = adverse_events_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.503390,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
1,1,0.629988,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
2,2,0.543418,0.509804,0.520000,0.500000,0.519231,0.480769,0.519231
3,3,0.579947,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
4,4,0.589782,0.625000,0.526316,0.769231,0.538462,0.461538,0.538462
5,5,0.676807,0.656250,0.552632,0.807692,0.576923,0.423077,0.576923
6,6,0.658677,0.657143,0.522727,0.884615,0.538462,0.461538,0.538462
7,7,0.656688,0.538462,0.538462,0.538462,0.538462,0.461538,0.538462
8,8,0.717157,0.646154,0.538462,0.807692,0.557692,0.442308,0.557692
9,9,0.702864,0.622222,0.736842,0.538462,0.673077,0.326923,0.673077


## Above results are for Adverse events data

In [31]:
results_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Results_hypothesis_evidences.csv')
print(results_df)

hypothesis_lst=results_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=results_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=results_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

results_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   there is a 13.2% difference between the result...   
1   the primary trial does not report the PFS or o...   
2   the shortest PFS in cohort 1 of the primary tr...   
3   the outcome measurement of the primary trial i...   
4   The the primary trial results section reports ...   
5   The the primary trial results section reports ...   
6   The shortest PFS in cohort 1 of the primary tr...   
7   the secondary trial and the primary trial both...   
8   there is a minimal difference between the resu...   
9   the primary trial and the secondary trial use ...   
10  at least 2 participants of the primary trial, ...   
11  the primary trial does not report the PFS or o...   
12  The results from the secondary trial and the p...   
13  neither of the interventions in the primary tr...   
14  The shortest PFS in cohort 1 of the primary tr...   
15  there were 6 recorded deaths in cohort 1 of th...   
16  the primary trial reports t

In [32]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(results_data)):
            batch_texts, batch_labels = results_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.529943,0.666667,0.520000,0.928571,0.535714,0.464286,0.535714
1,1,0.492687,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
2,2,0.516032,0.579710,0.487805,0.714286,0.482143,0.517857,0.482143
3,3,0.517777,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
4,4,0.548776,0.575758,0.500000,0.678571,0.500000,0.500000,0.500000
5,5,0.656860,0.597015,0.512821,0.714286,0.517857,0.482143,0.517857
6,6,0.687813,0.657534,0.533333,0.857143,0.553571,0.446429,0.553571
7,7,0.705771,0.603175,0.542857,0.678571,0.553571,0.446429,0.553571
8,8,0.740818,0.646154,0.567568,0.750000,0.589286,0.410714,0.589286
9,9,0.733778,0.565217,0.722222,0.464286,0.642857,0.357143,0.642857


## Above results are for Results data

In [34]:
eligibility_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Eligibility_hypothesis_evidences.csv')
print(eligibility_df)

hypothesis_lst=eligibility_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=eligibility_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=eligibility_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

eligibility_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Patients with significantly elevated ejection ...   
1   Prior treatment with fulvestrant or with a pho...   
2   only patients with a HER2-positive status can ...   
3   T4 N2 M4 patients are eligible for the primary...   
4   Patients must have a palpable carcinoma to be ...   
5   to be eligible for the primary trial patients ...   
6   ongoing flucytosine treatments are not permitt...   
7   Only patients with triple negative breast canc...   
8   adults and children can take part in the prima...   
9   Prior treatment with fulvestrant or with a pho...   
10  Presence of hot flashes for at least 4 month p...   
11  to be eligible for the primary trial candidate...   
12  A patient with a node positive T2 N2 M0 adenoc...   
13  ECOG score < 2 is necessary to be eligible for...   
14  patients with a Malignant brain tumour or colo...   
15  the secondary trial and the primary trial do n...   
16  all participants of the pri

In [35]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(eligibility_data)):
            batch_texts, batch_labels = eligibility_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.499670,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000
1,1,0.504530,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
2,2,0.607497,0.526316,0.517241,0.535714,0.517857,0.482143,0.517857
3,3,0.574442,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
4,4,0.558770,0.658228,0.509804,0.928571,0.517857,0.482143,0.517857
5,5,0.642515,0.683544,0.529412,0.964286,0.553571,0.446429,0.553571
6,6,0.692479,0.658537,0.500000,0.964286,0.500000,0.500000,0.500000
7,7,0.702168,0.626866,0.538462,0.750000,0.553571,0.446429,0.553571
8,8,0.672882,0.647887,0.534884,0.821429,0.553571,0.446429,0.553571
9,9,0.653382,0.476190,0.714286,0.357143,0.607143,0.392857,0.607143


## Above results are for Eligibility data

In [37]:
intervention_df=pd.read_csv('/kaggle/input/nlp-project/Training Data csv/Intervention_hypothesis_evidences.csv')
print(intervention_df)

hypothesis_lst=intervention_df['hypothesis'].values.tolist()
len(hypothesis_lst)

evidence_lst=intervention_df['evidences'].apply(lambda l:' '.join(json.loads(l))).values.tolist()
len(evidence_lst)

label2id={"Contradiction":0,"Entailment":1}
label_lst=intervention_df['label'].apply(lambda x:label2id[x]).values.tolist()
len(label_lst)

intervention_data=InputSequence(text_tok,hypothesis_lst,evidence_lst,label_lst,gpu=True)

                                           hypothesis  \
0   Both cohorts of the primary trial undergo a to...   
1   both cohorts of the primary trial receive iden...   
2   the primary trial participants recieve doses o...   
3   Cohort 1 and 2 of the primary trial do not rec...   
4   Participants of the primary trial will not rec...   
5   the primary trial participants receive more th...   
6   All patients in the primary trial receive high...   
7   the primary trial participants are administere...   
8   the primary trial and the secondary trial part...   
9   Cohort 1 and 2 of the primary trial do not rec...   
10  the primary trial participants are not adminis...   
11  All the primary trial participants are adminis...   
12  the primary trial participants are administere...   
13  All the primary trial participants are adminis...   
14  the primary trial participants are administere...   
15  80% of patients in the primary trial received ...   
16  the primary trial participa

In [38]:
import torch
import torch.nn.functional as F
import numpy as np
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import average_precision_score, f1_score, precision_score, recall_score, accuracy_score
import pandas as pd

# Define the faithfulness function
def faithfulness(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] != gold[key]["Causal_type"][1]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

# Define the consistency function
def consistency(predictions, gold):
    results = []
    for key in predictions.keys():
        # Adjust according to the actual structure of your data
        if predictions[key]["Prediction"] == gold[key]["Label"]:
            results.append(1)
        else:
            results.append(0)
    return sum(results) / len(predictions)

scores = []
model_names = ['dmis-lab/biobert-base-cased-v1.2'] + [
    '/kaggle/working/clf_models/dmis-lab/biobert-base-cased-v1.2_epoch_{}.pt'.format(format(epoch, '05d')) for epoch in range(10)
]

# Initialize containers for faithfulness and consistency scores
faithfulness_scores = []
consistency_scores = []

for model_name in model_names:
    scores.append([])
    clf = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()

    # Placeholder for storing predictions in the required format for faithfulness and consistency evaluations
    predictions = {}
    gold = {idx: {"Label": label, "Causal_type": ["", label]} for idx, label in enumerate(label_lst)}


    with torch.no_grad(), torch.cuda.amp.autocast(enabled=False):
        for batch in range(len(intervention_data)):
            batch_texts, batch_labels = intervention_data[batch]
            batch_size = 32
            num_batches = (len(batch_texts['input_ids']) + batch_size - 1) // batch_size

            batch_logits = []
            for i in range(num_batches):
                start = i * batch_size
                end = (i + 1) * batch_size
                input_ids = batch_texts['input_ids'][start:end]
                attention_mask = batch_texts['attention_mask'][start:end]
                logits = clf(input_ids=input_ids, attention_mask=attention_mask).logits
                batch_logits.append(logits.cpu())

            logits = torch.cat(batch_logits, dim=0)
            scores[-1].append(F.softmax(logits, dim=1).numpy())

            # Simulate prediction storage for faithfulness and consistency evaluation
            # This needs to be adapted to match your actual data structure
            for idx, logit in enumerate(logits):
                pred_label = torch.argmax(logit).item()
                predictions[batch * batch_size + idx] = {"Prediction": pred_label}
                # gold should be populated accordingly

    scores[-1] = np.concatenate(scores[-1], axis=0)
    clf.cpu()

    # Calculate faithfulness and consistency here
    faithfulness_scores.append(faithfulness(predictions, gold))
    consistency_scores.append(consistency(predictions, gold))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
y_true = label_lst
results = []
for epoch in range(len(scores)):
    y_prob = scores[epoch][:, 1]
    y_pred = [1 if a > 0.5 else 0 for a in y_prob]
    results.append([
        'pretrained' if epoch == 0 else epoch,
        average_precision_score(y_true, y_prob),
        f1_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        accuracy_score(y_true, y_pred),
        faithfulness_scores[epoch],  # Add faithfulness score
        consistency_scores[epoch]    # Add consistency score
    ])

columns = ['epoch', 'AVG_PREC', 'F1', 'PREC', 'REC', 'ACC', 'Faithfulness', 'Consistency']
pd.DataFrame(results, columns=columns)


,epoch,AVG_PREC,F1,PREC,REC,ACC,Faithfulness,Consistency
0,pretrained,0.521286,0.638298,0.517241,0.833333,0.527778,0.472222,0.527778
1,1,0.589758,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
2,2,0.530432,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
3,3,0.599219,0.666667,0.500000,1.000000,0.500000,0.500000,0.500000
4,4,0.574563,0.680000,0.531250,0.944444,0.555556,0.444444,0.555556
5,5,0.571859,0.636364,0.538462,0.777778,0.555556,0.444444,0.555556
6,6,0.613684,0.666667,0.533333,0.888889,0.555556,0.444444,0.555556
7,7,0.638155,0.711111,0.592593,0.888889,0.638889,0.361111,0.638889
8,8,0.679813,0.666667,0.555556,0.833333,0.583333,0.416667,0.583333
9,9,0.664490,0.516129,0.615385,0.444444,0.583333,0.416667,0.583333


## Above results are for Intervention data